In [2]:
import pickle
import time
import numpy as np
import pandas as pd

import torch
import torchTools as tt

## Import beamline

Using 29ID/IEX

In [3]:
from beamline_29 import IEX

## Importing and testing GA package

Developed by Rebecca Aloisio during 2020 summer internship with XSD/BC.  Full repo and testing can be found here: https://github.com/Automated-Beamline-Endeavors/GA4beamlines

Locally kept in ../GA4beamlines and not quite a "package" hence the clunky import process.

In [7]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/beams/MWYMAN/sandbox/GA4beamlines/')

import ga4beamlines as ga4b

Imported!
I've been imported!


### Coordinating motor definitions

GA4beamlines uses a dictionary for the motors and beamline_29 had it's own class attribute for the positions and ranges

### General GA setup

In [ ]:
nGenerations = 10
critVal = 0.9 # TODO what's this for beamline? 
startPop = None

popSize = 100
nElite = 10
alpha = 0.75
s = 1.5

In [ ]:
survivSelMode = ga4b.sMode[1]       # Genitor
parentSelMode = ga4b.pMode[0]       # ProbRank
crossOverMode = ga4b.cxMode[1]      # Simple
mutationMode = ga4b.mMode[1]        # Gaussian mutation
# fitnessMode = {"type": "Func", "name": funcnName} to be set in each section

## GA vs Shadow

First need to create fitness function based on shadow ouput

In [ ]:
def shadowFitness(x, beamline):
    '''
        x                  : array of positions for moveable DOF
        beamline           : shadow beamline model
    '''    
    
    #TODO add error checking to make sure x is same shape as where beamline.mask == 1
    
    beamline.pos[beamline.mask == 1] = x #need to expand to handle masking as x only covers the DOF but pos includes unmoved DOF
    fitness = beamline.run()

    return fitness

In [ ]:
shadowFitness(x,IEX)

In [ ]:
fitnessMode = {"type": "Func", "name": shadowFitness}

## GA vs Surrogate

First need to import scaling and model

In [ ]:
#import scaling
input_fn = 'IEX_100k_04w.pkl'

scaling_fn = input_fn.split('.')[0]+'_scaling.pkl'
with open(scaling_fn, 'rb') as f:
    scaling = pickle.load(f)

In [ ]:
#import model
nn_fn = input_fn.split('.')[0]+'_NN_results.pkl'
with open(nn_fn, 'rb') as f:
    result = pickle.load(f)

Need to create fitness function based on surrogate NN output

In [ ]:
def surrogateNNFitness(x, surrogate_model):
    '''
        x                  : array of positions that is the expected input for the surrogate model
        surrogate_model    : trained pyTorch model
    '''
    
    x_pd = pd.DataFrame(x, columns=['oe1x','oe1xrot','oe2x','oe2xrot','oe3xrot','oe4xrot','oe5y','oe5xrot','oe5yrot','oe6y','oe6xrot','oe6yrot',])
    xFeatures = scaling.transform(x_pd)
    xFeatures = pd.DataFrame(xFeatures, columns=x_pd.columns)

    x_tensor = torch.tensor(xFeatures.values) 
    
    model = surrogate_model
    device = torch.device('cpu')
    tt.to_device(model, device)

    fitness = model(tt.to_device(x_tensor.float(), device))
    
    return fitness
    

In [ ]:
surrogateNNFitness(test_pos, result[4]['model'])

In [ ]:
fitnessMode = {"type": "Func", "name": surrogateNNFitness}